In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sys
import os

from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from PIL import Image

In [14]:
class Conv(nn.Module):
    def __init__(self, out_channels, init_channels=3, input_size=6):
        super(Conv, self).__init__()

        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
                nn.Dropout2d(0.3)
            )
        
        self.conv_features = nn.Sequential(
            conv_block(init_channels, 32),
            conv_block(32, 64),
            conv_block(64, 128),
            nn.Conv2d(128, out_channels, kernel_size=1)
        )
        
        self.output_H = input_size // (2 * 2 * 2)
        self.output_W = input_size // (2 * 2 * 2)
    
    def forward(self, x):
        x = self.conv_features(x)
        return x

In [15]:
class MultiFeatureFusion(nn.Module):
    def __init__(self, input_size, embed_dim=256, hidden_dim=512, num_features=1, num_classes=3):
        super(MultiFeatureFusion, self).__init__()

        self.embed_dim = embed_dim
        dim_per_cnn = embed_dim // num_features

        if num_features == 1:
            self.cnn_f1 = Conv(input_size=input_size, out_channels=dim_per_cnn)
            
        elif num_features == 2:
            self.cnn_f1 = Conv(input_size=input_size, out_channels=dim_per_cnn)
            self.cnn_f2 = Conv(input_size=input_size, out_channels=dim_per_cnn)

        else:
            self.cnn_f1 = Conv(input_size=input_size, out_channels=dim_per_cnn)
            self.cnn_f2 = Conv(input_size=input_size, out_channels=dim_per_cnn)
            self.cnn_f3 = Conv(input_size=input_size, out_channels=dim_per_cnn)

        self.f_h = self.cnn_f1.output_H
        self.f_w = self.cnn_f1.output_W
    
        total_in_channels = dim_per_cnn * num_features
        self.fusion_block = nn.Sequential(
            nn.Conv2d(total_in_channels, hidden_dim, kernel_size=1),
            nn.ReLU(),
        )

        final_vec_size = hidden_dim * self.f_h * self.f_w

        self.classifier = nn.Sequential(
            nn.Linear(final_vec_size, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, num_classes)
        )
        
    def forward(self, f_1, f_2=None, f_3=None):
        f_list = []
        if f_2 == None and f_3 == None:
            f1 = self.cnn_f1(f_1)
            f_list.append(f1)

        elif f_2 != None and f_3 == None:            
            f1 = self.cnn_f1(f_1)
            f2 = self.cnn_f2(f_2)
            f_list.append(f1)
            f_list.append(f2)

        else:
            f1 = self.cnn_f1(f_1)
            f2 = self.cnn_f2(f_2)
            f3 = self.cnn_f3(f_3)
            f_list.append(f1)
            f_list.append(f2)
            f_list.append(f3)

        x = torch.cat(f_list, dim=1)
        x = self.fusion_block(x)
        x = x.flatten(start_dim=1)
        
        return self.classifier(x)
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [16]:
class MFCT_Net(nn.Module):
    def __init__(self, input_size, input_dim=48, embed_dim=384, hidden_dim=512, num_heads=8, num_layers=3, num_classes=2):
        super(MFCT_Net, self).__init__()

        self.embed_dim = embed_dim
        channel_per_cnn = embed_dim // 3

        self.cnn_gasf = Conv(input_size=input_size, out_channels=channel_per_cnn)
        self.cnn_gadf = Conv(input_size=input_size, out_channels=channel_per_cnn)
        self.cnn_rp = Conv(input_size=input_size, out_channels=channel_per_cnn)

        self.feature_h = self.cnn_gasf.output_H
        self.feature_w = self.cnn_gasf.output_W
        self.seq_len = self.feature_h * self.feature_w

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=num_heads, dim_feedforward=embed_dim * 4, dropout=0.1, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layer,
            num_layers=num_layers
        )

        self.positional_embedding = nn.Parameter(torch.randn(1, self.seq_len, embed_dim))

        self.fc_head = nn.Linear(embed_dim, num_classes)

        self.fc = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

    def forward(self, gasf, gadf, rp):
        f_gasf = self.cnn_gasf(gasf)
        f_gadf = self.cnn_gadf(gadf)
        f_rp = self.cnn_rp(rp)

        # Prepare to input into Transformer
        x = torch.cat([f_gasf, f_gadf, f_rp], dim=1)
        x = x.flatten(start_dim=2)
        x = x.transpose(1, 2)
        x = x + self.positional_embedding

        x = self.transformer_encoder(x)

        x = x.mean(dim=1)
        x = self.fc_head(x)
        # x = self.fc(x)a
        return x

In [17]:
input_size = 24

img1 = torch.randn(6, 3, input_size, input_size)
img2 = torch.randn(6, 3, input_size, input_size)
img3 = torch.randn(6, 3, input_size, input_size)

single_model = MultiFeatureFusion(input_size=input_size, num_features=1)
double_model = MultiFeatureFusion(input_size=input_size, num_features=2)
_multi_model = MFCT_Net(
    input_size=input_size,
    num_classes=3) # MultiFeatureFusion(input_size=input_size, num_features=3)

logits = single_model(img1)
print(logits.shape)
print(count_parameters(single_model))

logits = double_model(img1, img2)
print(logits.shape)
print(count_parameters(double_model))

logits = _multi_model(img1, img2, img3)
print(logits.shape)
print(count_parameters(_multi_model))

torch.Size([6, 3])
2619203
torch.Size([6, 3])
2712451
torch.Size([6, 3])
5854403


In [18]:
class NPZdataset(Dataset):
    def __init__(self, npz_path, transform=None):
        self.transform = transform
        data = np.load(npz_path)

        self.gasf_data = data['gasf']
        self.gadf_data = data['gadf']
        self.rp_data = data['rp']
        self.labels = data['labels']

        self.num_samples = len(self.labels)

        print(f"Loaded NPZ Dataset: {self.num_samples} samples.")

    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, index):
        img_gasf = transforms.ToPILImage()(self.gasf_data[index])
        img_gadf = transforms.ToPILImage()(self.gadf_data[index])
        img_rp = transforms.ToPILImage()(self.rp_data[index])
        
        label = self.labels[index]

        if self.transform:
            img_gasf = self.transform(img_gasf)
            img_gadf = self.transform(img_gadf)
            img_rp = self.transform(img_rp)

        return img_gasf, img_gadf, img_rp, torch.tensor(label, dtype=torch.long)

In [ ]:
npz_path = 'path'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomErasing()
])

dataset = datasets.ImageFolder(root='path', transform=transform) # NPZdataset(npz_path, transform)

total_size = len(dataset)
train_size = int(total_size * 0.7)
val_size = int(total_size * 0.15)
test_size = total_size - train_size - val_size

generator = torch.Generator().manual_seed(42)

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size], generator=generator
    )

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0
)

In [72]:
from torch import nn, optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

epochs = 30

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(single_model.parameters(), lr=1e-4, weight_decay= 1e-5)

train_logs, valid_logs = [], []

single_model.to(device)

MultiFeatureFusion(
  (cnn_f1): Conv(
    (conv_features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Dropout2d(p=0.3, inplace=False)
      )
      (1): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Dropout2d(p=0.3, inplace=False)
      )
      (2): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Dropout2d(p=0.3, inplace=False)
      )
      (3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (fusion_block): Sequential(
    (0): Conv2d(256, 512, kernel_size

In [ ]:
from tqdm import tqdm

for epoch in range(epochs):
    single_model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    progress_bar = tqdm(train_loader, desc=f"Train Epoch [{epoch + 1}/{epochs}]", leave=True)
    
    for images, labels in progress_bar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = single_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        acc = accuracy_score(all_labels, all_preds)
        prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)

        progress_bar.set_postfix({
            'Loss': f"{loss.item():.4f}",
            'Acc': f"{acc:.4f}",
            'Prec': f"{prec:.4f}",
            'Rec': f"{rec:.4f}"
        })

        avg_loss = running_loss / len(train_loader)
        final_acc = accuracy_score(all_labels, all_preds)
        final_prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        final_rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    train_logs.append({
        "Epoch": epoch + 1,
        "Train Loss": running_loss / len(train_loader),
        "Train Accuracy": final_acc,
        "Train Precision": final_prec,
        "Train Recall": final_rec
    })

    print(f"\nEpoch {epoch + 1}, Summary: Loss = {avg_loss:.4f}, Accuracy = {final_acc:.4f}, Precision = {final_prec:.4f}, Recall = {final_rec:.4f}\n", flush=True)

    single_model.eval()
    val_preds, val_labels = [], []
    val_loss = 0.0

    valid_progress_bar = tqdm(val_loader, desc=f"Validation Epoch [{epoch + 1}/epochs]", leave=True)

    with torch.no_grad():
        for image, label in valid_progress_bar:
            image = image.to(device)
            label = label.to(device)

            outputs = single_model(image)
            loss = criterion(outputs, label)
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(label.cpu().numpy())

            valid_progress_bar.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'Acc': f"{accuracy_score(val_labels, val_preds):.4f}",
                'Prec': f"{precision_score(val_labels, val_preds, average='macro', zero_division=0):.4f}",
                'Rec': f"{recall_score(val_labels, val_preds, average='macro', zero_division=0):.4f}"
            })

        val_acc = accuracy_score(val_labels, val_preds)
        val_prec = precision_score(val_labels, val_preds, average='macro', zero_division=0)
        val_rec = recall_score(val_labels, val_preds, average='macro', zero_division=0)
        avg_val_loss = val_loss / len(val_loader)
        print(f"✅ Validation Summary: Loss = {avg_val_loss:.4f}, Acc = {val_acc:.4f}, Prec = {val_prec:.4f}, Rec = {val_rec:.4f}\n")
        
        valid_logs.append({
            "Epoch": epoch + 1,
            "Val Loss": val_loss / len(val_loader),
            "Val Accuracy": val_acc,
            "Val Precision": val_prec,
            "Val Recall": val_rec
        })

In [ ]:
test_preds, test_labels, test_results = [], [], []
test_loss = 0.0
    
print("***** Start testing the model")
test_progress_bar = tqdm(test_loader, desc=f"Testing the model", leave=True)
with torch.no_grad():
    for images, labels in test_progress_bar:
        images, labels = images.to(device), labels.to(device)

        outputs = single_model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_acc = accuracy_score(test_labels, test_preds)
test_prec = precision_score(test_labels, test_preds, average='macro', zero_division=0)
test_rec = recall_score(test_labels, test_preds, average='macro', zero_division=0)
test_results.append({
    "Test Accuracy": test_acc,
    "Test Precision": test_prec,
    "Test Recall": test_rec
})

print(f"\n🏁 Test Set Evaluation Result:")
print(f"Loss = {test_loss / len(test_loader):.4f}, Accuracy = {test_acc:.4f}, Precision = {test_prec:.4f}, Recall = {test_rec:.4f} ")


***** Start testing the model


Testing the model: 100%|██████████| 23/23 [00:11<00:00,  1.99it/s]


🏁 Test Set Evaluation Result:
Loss = 1.0668, Accuracy = 0.4266, Precision = 0.4285, Recall = 0.4284 
